Need to break down donors into the following dimensions

- onetime, recurring, events, donor logins, registrants (fundraisers)
- source (don form, mobile, sms, p2p, givi)

In [22]:
from datetime import datetime
print("{:%Y-%m-%d %H:%M}".format(datetime.now()))
print(datetime.now())

2020-12-10 02:00
2020-12-10 02:00:22.991121


In [2]:
import pandas as pd
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

In [3]:
q = "select * from transactions where status='A'"
df = redshift_query_read(q)

In [8]:
cols = [c for c in df.columns if '_amt' in c]
df[cols].tail(3)

,donations_amt,purchases_amt,events_amt,registrations_amt,smspledge_amt
8727267,50.0,0.0,0.0,0.0,0.0
8727268,0.0,0.0,40.0,0.0,0.0
8727269,25.0,0.0,0.0,0.0,0.0


In [7]:
onetime_msk = (df['donations_amt']>0)&(df['recurring']==0)&(df['events_amt']==0)
onetime_msk = onetime_msk&(df['purchases_amt']==0)&(df['events_amt']==0)&(df['registrations_amt']==0)
onetime_donors = len(df[onetime_msk]['email'].unique())
recurring_donors = len(df[df['recurring']!=0]['email'].unique())
events_donors = len(df[df['events_amt']>0]['email'].unique())

"{:,}".format(onetime_donors), "{:,}".format(recurring_donors), "{:,}".format(events_donors)

('1,905,412', '171,717', '454,771')

In [10]:
df['source'].unique()

array(['don_form', 'vt', 'p2p', 'kiosk', 'mobile', 'sms', 'mobilevt',
       'fb', 'givi'], dtype=object)

In [11]:
donor_data = []
for donor in df['email'].unique().tolist():
    _df = df[df['email']==donor]
    donor_data.append({
        'donor': donor,
        'transactions': len(_df),
        'total_amount': _df['amount'].sum(),
        'avg_amount': _df['amount'].mean(),
        'donations_amt': _df['donations_amt'].sum(),
        'donations_count': _df['donations_count'].sum(),
        'purchases_amt': _df['purchases_amt'].sum(),
        'purchases_count': _df['purchases_quantity'].sum(),
        'events_amt': _df['events_amt'].sum(),
        'events_tickets': _df['events_tickets'].sum(),
        'registrations_amt': _df['registrations_amt'].sum(),
        'registrations_count': _df['registrations_count'].sum(),
        'smspledge_amt': _df['smspledge_amt'].sum(),
        'smspledge_count': _df['smspledge_amt'].sum(),
        'don_form_count': len(_df[_df['source']=='don_form']), 
        'don_form_amount': _df[_df['source']=='don_form']['amount'].sum(), 
        'vt_count': len(_df[_df['source']=='vt_count']), 
        'vt_amount': _df[_df['source']=='vt']['amount'].sum(),
        'p2p_count': len(_df[_df['source']=='p2p_count']), 
        'p2p_amount': _df[_df['source']=='p2p']['amount'].sum(),
        'kiosk_count': len(_df[_df['source']=='kiosk_count']), 
        'kiosk_amount': _df[_df['source']=='kiosk']['amount'].sum(),
        'mobile_count': len(_df[_df['source']=='mobile_count']), 
        'mobile_amount': _df[_df['source']=='mobile']['amount'].sum(),
        'sms_count': len(_df[_df['source']=='sms_count']), 
        'sms_amount': _df[_df['source']=='sms']['amount'].sum(),
        'mobilevt_count': len(_df[_df['source']=='don_form']),
        'mobilevt_amount': _df[_df['source']=='mobilevt']['amount'].sum(),
        'fb_count': len(_df[_df['source']=='fb_count']), 
        'fb_amount': _df[_df['source']=='fb']['amount'].sum(),
        'givi_count': len(_df[_df['source']=='givi_count']),
        'givi_amount': _df[_df['source']=='givi']['amount'].sum()
    })
    if len(donor_data) % 100000 == 0:
        print("done with {} donors".format(len(donor_data)))

KeyboardInterrupt: 

In [14]:
len(donor_data)

29758